# Web Scaping

## Scraping a single page - Tests

In [4]:
import requests
from bs4 import BeautifulSoup
from dateutil import parser 
import textstat

In [5]:
#This is just to make it so the request goes through and the Dawson website does not decline it 
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'referrer': 'https://google.com'
}

In [6]:
# URL of program page for pure and applied
url = 'https://www.dawsoncollege.qc.ca/pure-applied/'
r = requests.get(url, headers=headers)

In [8]:
# Status code is 200 means all is good and request went through
r.status_code

200

In [10]:
# Get HTML of the page
html = r.text.strip()

In [13]:
# Make html of page a BeautifulSoup 
soup = BeautifulSoup(html, 'lxml')

In [24]:
# Header Content of page
content = soup.find(class_='entry-content')
#Get program title and strip any white spaces
programTitle = content.find('h2').contents[0].strip()

Pure & Applied Science (200.P2)


In [38]:
# Get when last modified
dateModified = soup.find(class_='page-mod-date').contents[0].strip()
# Remove the 'Last Modified: ' field so just date is left
dateModified = dateModified[15:]
print(dateModified)

August 8, 2018


# Function to parse a single page

In [4]:
# Parse a single Program page at Dawson College
def parseProgramPage(url):
    
    #This is just to make it so the request goes through and the Dawson website does not decline it 
    headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'referrer': 'https://google.com'
    }
    
    # URL of program page for pure and applied
    url = 'https://www.dawsoncollege.qc.ca/pure-applied/'
    r = requests.get(url, headers=headers)
    
    # Get HTML of the page
    html = r.text.strip()
    
    # Make html of page a BeautifulSoup 
    soup = BeautifulSoup(html, 'lxml')
    
    programName = getName(soup)
    programDateMod = getDateOfMods(soup)
    readingLevel = getReadingLevel(soup)
    
    #Page data to be returned
    pageData = {
        'programName': programName,
        'date': programDateMod,
        'readingLevel': readingLevel
    }
    
    return pageData

# Get reading level 
def getReadingLevel(soup):
    content = soup.find(class_='entry-content')
    readingLevel = textstat.flesch_kincaid_grade(content.text)
    return reading_level
    
# Used to get name of program
def getName(soup):
    # Main content of page
    content = soup.find(class_='entry-content')
    #Get program title and strip any white spaces
    programTitle = content.find('h2').contents[0].strip()
    return programTitle

#Get the date of when the page was last modified
def getDateOfMods(soup):
    # Get when last modified
    dateModified = soup.find(class_='page-mod-date').contents[0].strip()
    # Remove the 'Last Modified: ' field so just date is left
    dateModified = dateModified[15:]
    return dateModified
    